In [8]:
from netCDF4 import Dataset
import glob as glob

In [9]:
# compute annual runoff by differencing the first and last accumulation surfaces

years = np.arange(2002,2014) # water years 2002-2013

for year in years:
    strtfl = '/Volumes/data/wrf/%s/wrf2d_d01_%s-09-30_thinned_crop.nc'%(year-1,year-1)
    endfl = '/Volumes/data/wrf/%s/wrf2d_d01_%s-09-30_thinned_crop.nc'%(year,year)
    
    start = Dataset(strtfl,'r')
    end = Dataset(endfl,'r')
    
    gw = np.array(end.variables['UDROFF']) - np.array(start.variables['UDROFF'])
    ro = np.array(end.variables['SFROFF']) - np.array(start.variables['SFROFF'])
    
    np.savez_compressed('./data/runoff_wy%s.npz'%(year),ro)
    np.savez_compressed('./data/groundwater_wy%s.npz'%(year),gw)
    
    print year
    

2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013


In [3]:
## Processed Clipped Model Output

years = np.arange(2002,2014) # water years 2002-2013

for year in years:

    days = pd.date_range(start='10-01-'+str(year-1),end='09-30-'+str(year),freq='D')

    files = []
    for d in days: #loop through days and append to file list
        files.append('/Volumes/data/wrf/%s/wrf2d_d01_%s_thinned_crop.nc'%(d.year,d.strftime('%Y-%m-%d')))

    n = len(files) # number of files
    d,h,w = np.array(Dataset(files[0],'r').variables['QSNBOT']).shape # get the shape of the array

    snowmelt = np.empty((n,h,w)) # preallocate numpy arrays
    #runoff = np.empty((1,h,w))
    #groundwater = runoff.copy()
    
    
    # preload the time slice before for accumulated variables
    #gw = np.array(Dataset('/Volumes/data/wrf/%s/wrf2d_d01_%s-09-30_thinned_crop.nc'%(year-1,year-1)).variables['UDROFF'])
    #ro = np.array(Dataset('/Volumes/data/wrf/%s/wrf2d_d01_%s-09-30_thinned_crop.nc'%(year-1,year-1)).variables['SFROFF'])

    ct = 0
    for fl in files:
        tmp = Dataset(fl,'r')        
        snowmelt[ct] = np.array(tmp.variables['QSNBOT']) * (60.*60.*24.) # mm/s >> mm/minute >> mm/hour >> mm/day
        #runoff += (np.array(tmp.variables['SFROFF'])-ro) # mm/day
        #groundwater += (np.array(tmp.variables['UDROFF'])-gw) # mm/day
        ct += 1 # increment counter
        #gw = np.array(tmp.variables['UDROFF'])
        #ro = np.array(tmp.variables['SFROFF'])
        
    # compute averages 
    
    #runoff /= ct # mm/day
    #groundwater /= ct # mm/day
    print ct
    np.savez_compressed('./data/snowmelt_wy%s.npz'%(year),snowmelt)
    #np.savez_compressed('./data/runoff_wy%s.npz'%(year),runoff)
    #np.savez_compressed('./data/groundwater_wy%s.npz'%(year),groundwater)
    print 'Processed Water Year %s'%(year)

365
Processed Water Year 2002
365
Processed Water Year 2003
366
Processed Water Year 2004
365
Processed Water Year 2005
365
Processed Water Year 2006
365
Processed Water Year 2007
366
Processed Water Year 2008
365
Processed Water Year 2009
365
Processed Water Year 2010
365
Processed Water Year 2011
366
Processed Water Year 2012
365
Processed Water Year 2013


In [15]:
## Processed Clipped Precip Data

years = np.arange(2002,2014) # water years 2002-2013
for year in years:
    days = pd.date_range(start='10-01-'+str(year-1),end='09-30-'+str(year),freq='M')
    
    files = []
    
    for d in days: #loop through days and append to file list
        files.append('/Volumes/data/wrf/precip/daily/wrf2d_d01_daily_tot_%s_crop.nc'%(d.strftime('%Y%m')))
    
    n = len(pd.date_range(start='10-01-'+str(year-1),end='09-30-'+str(year),freq='D'))
    d,h,w = np.array(Dataset(files[0],'r').variables['ACRAINLSM']).shape
    ct1 = 0 # counter 1
    ct2 = 0 # counter 2
    rain = np.empty((n,h,w)) # preallocate numpy array
    precip = np.empty((1,h,w))
    
    for fl in files:
        #print ct2
        tmp = Dataset(fl,'r')
        tmp2 = np.array(tmp.variables['ACRAINLSM']) # pull out rain
        n,h,w = tmp2.shape # get rain dimensions
        rain[ct2:ct2+n,:,:] = tmp2 # add rain to the preallocated array
        precip += np.array(tmp.variables['PREC_ACC_NC']).sum(axis=0) # add total monthly precip to the precip data frame
        ct1 += 1 # increment counter
        ct2 += n # add the length of the month to the second counter

    # compute averages 
    #precip /= ct2 # divide by number of days to get mm/day
    #print ct2
    np.savez_compressed('./data/rain_wy%s'%(year),rain)
    np.savez_compressed('./data/precip_wy%s'%(year),precip)
    print year

2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013


In [10]:
# Load the cell indexer
indexer = np.load('./data/index.npy')

In [11]:
# define a function to convert the numpy data files to a giant data frame
def load_data(wflux,surf,base,rain,snowmelt,precip,year,idx):
    '''
    wflux = water flux (rain + snowmelt)
    surf = surface runoff
    base = baseflow
    year = water year
    idx = indexer
    
    Function to reshape input data and append it onto the larger data frame
    
    '''
    
    m,p = np.shape(wflux)
    wflux = np.reshape(wflux,(m*p))
    surf = np.reshape(surf,(m*p))
    base = np.reshape(base,(m*p))
    rain = np.reshape(rain,(m*p))
    snowmelt = np.reshape(snowmelt,(m*p))
    precip = np.reshape(precip,(m*p))
    year = np.repeat(year,m*p)
    idx = np.reshape(idx,(m*p))
    
    # return a data frame of the input data to be appended onto the mother data frame
    return pd.DataFrame({'idx':idx,'wyear':year,'wflux':wflux,'runoff':surf,'baseflow':base,
                         'rain':rain,'snowmelt':snowmelt,'precip':precip})
    
    

In [12]:
# numpy files to data frame
wateryears = np.arange(2002,2014) 

df = pd.DataFrame()
for year in wateryears:
    wflux = np.load('./data/waterflux_wy%s.npy'%year)
    surf = np.load('./data/runoff_wy%s.npz'%year)['arr_0']
    base = np.load('./data/groundwater_wy%s.npz'%year)['arr_0']
    rain = np.load('./data/processedrain_wy%s.npy'%year)
    snowmelt = np.load('./data/processedsnowmelt_wy%s.npy'%year)
    precip = np.load('./data/precip_wy%s.npz'%year)['arr_0']
    df = df.append(load_data(wflux,surf,base,rain,snowmelt,precip,year,indexer))
    print year

2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013


In [13]:
df.to_pickle('./data/wrf_data.df') # save the data frame